In [40]:
!conda env list

# conda environments:
#
base                  *  /opt/anaconda3
django_1                 /opt/anaconda3/envs/django_1
tsr                      /opt/anaconda3/envs/tsr
tsr2                     /opt/anaconda3/envs/tsr2



In [42]:
!pip install pandas

In [44]:
#CSV変換等に必要なライブラリをインポート
!pip install tabula-py
!pip install pdfminer.six
!pip install PyPDF2
!pip install mojimoji
!pip install jeraconv
!pip install tqdm
!pip install PyDrive
!pip install joblib
#!pip --proxy url:port pdfminer.six
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LAParams
from io import StringIO
from tqdm import tqdm
from dateutil.relativedelta import relativedelta
from jeraconv import jeraconv
from PyPDF2 import PdfReader
from tabula.io import read_pdf

import os
import pandas as pd
import tabula
import warnings
import mojimoji
import re
import csv

warnings.simplefilter('ignore')

In [46]:
pip list

Package                           Version
--------------------------------- ------------
aext-assistant                    4.0.15
aext-assistant-server             4.0.15
aext-core                         4.0.15
aext-core-server                  4.0.15
aext-panels                       4.0.15
aext-panels-server                4.0.15
aext-share-notebook               4.0.15
aext-share-notebook-server        4.0.15
aext-shared                       4.0.15
agate                             1.7.1
agate-dbf                         0.2.2
agate-excel                       0.2.5
agate-sql                         0.5.8
aiobotocore                       2.7.0
aiohttp                           3.9.3
aioitertools                      0.7.1
aiosignal                         1.2.0
alabaster                         0.7.12
altair                            5.0.1
anaconda-anon-usage               0.4.3
anaconda-catalogs                 0.2.0
anaconda-client                   1.12.3
anaconda-cloud-auth 

In [48]:
os.getcwd()

'/Users/akamine_saki/Projects/notebook'

In [50]:
import tabula
print(tabula.__file__)

/opt/anaconda3/lib/python3.11/site-packages/tabula/__init__.py


In [52]:
read_path = "/Users/akamine_saki/Projects/notebook/TSR"

In [54]:
# #上記パスのフォルダ内を表示もともとあったコード
# dir = os.listdir(read_path)
# pdf_dir = dir[0]
# path = f'{read_path}\\{pdf_dir}'
# dir_ = os.listdir(path)
# dir_

In [56]:
# フォルダ内の最初のディレクトリを取得
dir_list = os.listdir(read_path)
pdf_dir = dir_list[0]  # 最初のディレクトリ

# フルパスを作成
path = os.path.join(read_path, pdf_dir)

# フォルダ内のファイルやディレクトリを表示
dir_ = os.listdir(path)
dir_  # これが最終的な結果

['.DS_Store', '福岡県_(糸島市,那珂川市)_349件.pdf']

In [58]:
pattern = '.*.pdf'

files = [f for f in dir_ if re.match(pattern,f)]

In [60]:
def read_pdf_by_row(f, page):
    warnings.simplefilter("ignore")
    columns = ['企業コード(TSR)','上場区分','TSR調査年月日','企業名ﾌﾘｶﾞﾅ（半角）','代表者氏名ﾌﾘｶﾞﾅ（半角）','会社名','代表者氏名','郵便番号','所在地','電話番号','設立年月日','創業年月日','資本金（千円単位）','従業員数','業種1','業種2','業種3',
                   '営業種目','営業所・支店住所','役員','仕入先','株主構成','販売先','財務情報1_時期','財務情報1_売上(千円)','財務情報1_純利益(千円)','財務情報2_時期','財務情報2_売上(千円)','財務情報2_純利益(千円)','財務情報3_時期','財務情報3_売上(千円)','財務情報3_純利益(千円)',
                    '取引銀行','売上伸長率','利益伸長率','TSR備考','代表者住所','生年月日','出身地','出身校']
    
    df_ = pd.DataFrame(columns = columns)
    df_.loc[0] = columns 

    print
    
    try:
        #df = tabula.read_pdf(f,encoding = 'utf-8',lattice=True, pages=page)
        df = read_pdf(f,encoding = 'utf-8',lattice=True, pages=page)
        #df = read_pdf(f, encoding='cp932', lattice=True, pages=page)
        #print(df)
        value0 = df[0].columns[0] #企業コード　TSRID__c
        value1 = df[1].columns[1][2:] #上場区分 Listing_classification__c
        value2 = df[2].columns[0] #調査年月日
        value3 = mojimoji.zen_to_han(df[3].columns[0]) #商号（カナ）KANA__C
        value4 = mojimoji.zen_to_han(df[4].columns[0]) #代表者カナ　PRESIDENTKANA__C
        value5 = df[5].columns[0] #商号（漢字）NAME
        value6 = df[6].columns[0] #代表者氏名　PRESIDENTNAME__C
        # value6_1  = value6.split(' ')[0] #姓
        # value6_2  = value6.split(' ')[1] #名
        
        value7 = df[7].columns[0].strip('〒')  #郵便番号 BILLINGPOSTALCODE
        value8 = df[8].columns[1] #所在地
        value9 = df[9].columns[0] #電話番号 PHONE
        value10 = df[10].columns[1] #設立年月 STARTDATE__C
        value11 = df[11].columns[1] #創業年月
        value12 = df[12].columns[0] #資本金 CAPITAL__C
        value13 = df[13].columns[0].strip('人') #従業員数赤嶺追加
        # value14 = df[14]  #業種
        value14_1 = df[14].columns[0] #業種1
        value14_2 = df[14].iloc[0][0] #業種2
        value14_3 = df[14].iloc[1][0] #業種3

        value15 = df[15].columns[0].replace('\r','') #営業種目
        value16 = df[16].columns[1].replace('\r','') #営業所・支店
        value17 = df[17].columns[0].replace('\r','') #役員
        value18 = df[18].columns[1].replace('\r','') #仕入先
        value19 = df[19].columns[0].replace('\r','') #大株主
        value20 = df[20].columns[1].replace('\r','') #販売先

        # value21 = df[21] #業績
        value21_1_RECYM = df[21].iloc[0][0] #期間1
        value21_1_AMOUNT = df[21].iloc[0][1] #売上1
        value21_1_PROFIT = df[21].iloc[0][4] #利益1
        value21_2_RECYM = df[21].iloc[1][0] #期間2
        value21_2_AMOUNT = df[21].iloc[1][1] #売上2
        value21_2_PROFIT = df[21].iloc[1][4] #利益2
        value21_3_RECYM = df[21].iloc[2][0] #期間3
        value21_3_AMOUNT = df[21].iloc[2][1] #売上3
        value21_3_PROFIT = df[21].iloc[2][4] #利益3

        value22 = df[22].columns[1].replace('\r','') #取引銀行
        value23 = df[23].columns[1].replace('\r','') #売上伸長率
        value24 = df[24].columns[0] #利益伸長率
        value25 = df[25].columns[1].replace('\r','') #概況
      # value26 = df[26].columns[0].replace('\r','') #代表者
        value27 = df[27].columns[0].replace('\r','') #代表者住所
        value28 = df[28].columns[0].replace('\r','')[:10] #生年月日
      # value29 = df[29].columns[0].replace('\r','') #干支
        value30 = df[30].columns[0].replace('\r','')[2:] #出身地
        value31 = df[31].columns[1].replace('\r','') #出身校

        #値をリスト化
        value = [value0,value1,value2,value3,value4,value5,value6,value7,value8,value9,value10,value11,value12,value13,value14_1,value14_2,value14_3,
                value15,value16,value17,value18,value19,value20,value21_3_RECYM,value21_3_AMOUNT,value21_3_PROFIT,value21_2_RECYM,value21_2_AMOUNT,value21_2_PROFIT,value21_1_RECYM,value21_1_AMOUNT,value21_1_PROFIT,
                value22,value23,value24,value25,value27,value28,value30,value31]
        # pdf_records.append(value)←ここで追加されているのでどちらがいいのか前田さんに確認
        

        #テーブルを作成
        df = pd.DataFrame([value],columns = columns)
        #テーブルに追加
        #df_ = df_.append(df,ignore_index = True)
        df_ = pd.concat([df_, df], ignore_index=True)
        #print(f'{f}を処理中({startpdf+t}/{endpdf})')
        #CSVへ変換
        #try :
        #   df_.to_csv(f'{f}_csv.csv',encoding='cp932')
        #except :
        #   df_.to_csv(f'{f}_csv.csv',encoding='utf-8')
        return value
    except Exception as e:
        print('表示するページはありません!')
        print(e)

In [62]:
# print(df_)

In [64]:
print(files)

['福岡県_(糸島市,那珂川市)_349件.pdf']


In [66]:
pdf_records = []
for f in files:
    full_path = os.path.join('TSR', '2024', f)
    # full_path = os.path.abspath(f)  # 絶対パスを取得
    
    print(f'{full_path}を読み込み')

    try:
        reader = PdfReader(full_path)
        print(f'ページ数: {len(reader.pages)}')

        start = 1
        # end = min(10, len(reader.pages))
        end = len(reader.pages)+1
        pages =range(start, end)
        
        for page in pages:
            print(f'ページ {page} を処理中')  
            pdf_records.append(read_pdf_by_row(full_path, page))
            
    except FileNotFoundError:
        print(f'ファイルが見つかりません: {full_path}')
    except Exception as e:
        print(f'エラーが発生しました: {e}')



# tabulaとtabula-pyの名前がバッティングしているのでは？
#そもそものテーブルの形になってない？

TSR/2024/福岡県_(糸島市,那珂川市)_349件.pdfを読み込み
ページ数: 349
ページ 1 を処理中


Got stderr: 11月 05, 2024 2:28:16 午後 org.apache.fontbox.ttf.CmapSubtable processSubtype14
警告: Format 14 cmap table is not supported and will be ignored



ページ 2 を処理中


Got stderr: 11月 05, 2024 2:28:17 午後 org.apache.fontbox.ttf.CmapSubtable processSubtype14
警告: Format 14 cmap table is not supported and will be ignored



ページ 3 を処理中


...

Got stderr: 11月 05, 2024 2:30:37 午後 org.apache.fontbox.ttf.CmapSubtable processSubtype14
警告: Format 14 cmap table is not supported and will be ignored



ページ 348 を処理中


Got stderr: 11月 05, 2024 2:30:38 午後 org.apache.fontbox.ttf.CmapSubtable processSubtype14
警告: Format 14 cmap table is not supported and will be ignored



ページ 349 を処理中


Got stderr: 11月 05, 2024 2:30:38 午後 org.apache.fontbox.ttf.CmapSubtable processSubtype14
警告: Format 14 cmap table is not supported and will be ignored



In [120]:
# #上記path内のディレクトリを取得
# startpdf = 0
# endpdf = 1
# for i in tqdm(range(startpdf,endpdf)):
#     dir = os.listdir(read_path)
#     dirs = []
#     pdf_dir = dir[i]
#     path = f'{read_path}\\{pdf_dir}'

#     dir_ = os.listdir(path)
#     dirs.append(dir_)
    
#     pattern = '.*.pdf'
#     filelist_copy = []

#     for i in range(0,len(dirs)):
#         for files in dirs[i]:
#             if re.match(pattern,files) :
#                 csvfile = files
#                 filelist_copy.append(csvfile)
#             else :
#                 csvfile = ''
#         print(path)
#         print(pdf_dir)
#         print(filelist_copy)

#     # "PDF"を順次読み込み
#     for t in range(0,len(filelist_copy)):
# #     for t in range(0,2):
#         print(f'{filelist_copy[t]}を読み込み')

#         os.chdir(path)
#         reader = PdfReader(filelist_copy[t])
#         #ページ数を取得する
#         print(len(reader.pages))

#         #スタートページを指定
#         start = 1
#         #ラストページを指定
#         end = len(reader.pages)

#         columns = ['企業コード(TSR)','上場区分','TSR調査年月日','企業名ﾌﾘｶﾞﾅ（半角）','代表者氏名ﾌﾘｶﾞﾅ（半角）','会社名','代表者氏名','郵便番号','所在地','電話番号','設立年月日','創業年月日','資本金（千円単位）','従業員数','業種1','業種2','業種3',
#                    '営業種目','営業所・支店住所','役員','仕入先','株主構成','販売先','財務情報1_時期','財務情報1_売上(千円)','財務情報1_純利益(千円)','財務情報2_時期','財務情報2_売上(千円)','財務情報2_純利益(千円)','財務情報3_時期','財務情報3_売上(千円)','財務情報3_純利益(千円)',
#                    '取引銀行','売上伸長率','利益伸長率','TSR備考','代表者住所','生年月日','出身地','出身校']
#         df_ = pd.DataFrame(columns = columns)

#         #"ページ"を順次読み込み
#         pages = range(start,end+1)
# #         pages = range(start,2)

#         for page in tqdm(pages):
#           try:
#             df = tabula.read_pdf(filelist_copy[t],encoding = 'cp932',lattice=True, pages=page)
#             value0 = df[0].columns[0] #企業コード　TSRID__c
#             value1 = df[1].columns[1][2:] #上場区分 Listing_classification__c
#             value2 = df[2].columns[0] #調査年月日 
#             value3 = mojimoji.zen_to_han(df[3].columns[0]) #商号（カナ）KANA__C
#             value4 = mojimoji.zen_to_han(df[4].columns[0]) #代表者カナ　PRESIDENTKANA__C
#             value5 = df[5].columns[0] #商号（漢字）NAME
#             value6 = df[6].columns[0] #代表者氏名　PRESIDENTNAME__C
#             # value6_1  = value6.split(' ')[0] #姓　
#             # value6_2  = value6.split(' ')[1] #名

            # value7 = df[7].columns[0].strip('〒')  #郵便番号 BILLINGPOSTALCODE
            # value8 = df[8].columns[1] #所在地
            # value9 = df[9].columns[0] #電話番号 PHONE
            # value10 = df[10].columns[1] #設立年月 STARTDATE__C
            # value11 = df[11].columns[1] #創業年月
            # value12 = df[12].columns[0] #資本金 CAPITAL__C
            # value13 = df[13].columns[0] #従業員数

            # # value14 = df[14]  #業種
            # value14_1 = df[14].columns[0] #業種1
            # value14_2 = df[14].iloc[0][0] #業種2
            # value14_3 = df[14].iloc[1][0] #業種3

            # value15 = df[15].columns[0].replace('\r','') #営業種目
            # value16 = df[16].columns[1].replace('\r','') #営業所・支店
            # value17 = df[17].columns[0].replace('\r','') #役員
            # value18 = df[18].columns[1].replace('\r','') #仕入先
            # value19 = df[19].columns[0].replace('\r','') #大株主
            # value20 = df[20].columns[1].replace('\r','') #販売先

          #   # value21 = df[21] #業績
          #   value21_1_RECYM = df[21].iloc[0][0] #期間1
          #   value21_1_AMOUNT = df[21].iloc[0][1] #売上1
          #   value21_1_PROFIT = df[21].iloc[0][4] #利益1
          #   value21_2_RECYM = df[21].iloc[1][0] #期間2
          #   value21_2_AMOUNT = df[21].iloc[1][1] #売上2
          #   value21_2_PROFIT = df[21].iloc[1][4] #利益2
          #   value21_3_RECYM = df[21].iloc[2][0] #期間3
          #   value21_3_AMOUNT = df[21].iloc[2][1] #売上3
          #   value21_3_PROFIT = df[21].iloc[2][4] #利益3

          #   value22 = df[22].columns[1].replace('\r','') #取引銀行
          #   value23 = df[23].columns[1].replace('\r','') #売上伸長率
          #   value24 = df[24].columns[0] #利益伸長率
          #   value25 = df[25].columns[1].replace('\r','') #概況
          # # value26 = df[26].columns[0].replace('\r','') #代表者
          #   value27 = df[27].columns[0].replace('\r','') #代表者住所
          #   value28 = df[28].columns[0].replace('\r','')[:10] #生年月日
          # # value29 = df[29].columns[0].replace('\r','') #干支
          #   value30 = df[30].columns[0].replace('\r','')[2:] #出身地
          #   value31 = df[31].columns[1].replace('\r','') #出身校

#             #値をリスト化
#             value = [value0,value1,value2,value3,value4,value5,value6,value7,value8,value9,value10,value11,value12,value13,value14_1,value14_2,value14_3,
#                     value15,value16,value17,value18,value19,value20,value21_3_RECYM,value21_3_AMOUNT,value21_3_PROFIT,value21_2_RECYM,value21_2_AMOUNT,value21_2_PROFIT,value21_1_RECYM,value21_1_AMOUNT,value21_1_PROFIT,
#                     value22,value23,value24,value25,value27,value28,value30,value31]

# #             #テーブルを作成
# #             df = pd.DataFrame([value],columns = columns)
# #             #テーブルに追加
# #             df_ = df_.append(df,ignore_index = True)
# #             print(f'{filelist_copy[t]}を処理中({startpdf+t}/{endpdf})')
# #             #CSVへ変換
# #             try :
# #                 df_.to_csv(f'{filelist_copy[t]}_csv.csv',encoding='cp932')
# #             except :
# #                 df_.to_csv(f'{filelist_copy[t]}_csv.csv',encoding='utf-8')
# #             except:
# #                 print('表示するページはありません')
# # #         os.chdir('C:\\Users\\ya\\OneDrive - 株式会社M＆Aバリューエンジニアリング\\共有用\\2021年下半期(50万社)\\未上場\\CSVフォルダ')
# #         df_.to_csv(f'{filelist_copy[t]}_csv.csv',encoding='cp932')
#                 return value
#                 except Exception as e:
#                     print('表示するページはありません!')
#                     print(e)

In [68]:
pdf_records

[['872236943',
  '未上場',
  '2023/06/05',
  'ｲﾜﾄｹﾝｾﾂ',
  'ﾆﾉﾐﾔ ﾊｼﾞﾒ',
  '岩戸建設(株)',
  '二宮 肇',
  '811-1224',
  '福岡県那珂川市安徳522-1',
  '092-555-4344',
  '2009/01',
  '2007/01',
  '5,100 千円',
  '5 ',
  '0621 土木工事業',
  nan,
  nan,
  '土木工事',
  'Unnamed: 1',
  '(取)平島洋一,古賀浩美',
  'Unnamed: 1',
  '古賀義美(500株),二宮肇(10株)',
  '別府土建,福岡県',
  '2022/09',
  '217,707',
  37,
  '2021/09',
  '124,967',
  178,
  '2020/09',
  '308,985',
  290,
  '福岡(春日原)',
  '174%',
  '21%',
  '2007年創業の土木工事業者で官公庁を含め実績は浸透しているが、低調な収益推移をたどり、財務基盤は構築途上にある。',
  '〒811-1255福岡県那珂川市恵子5-1-9スカイハイツ1号',
  '1962/11/15',
  'named: 0',
  'Unnamed: 1'],
 ['870092898',
  '未上場',
  '2023/09/15',
  'ｵｶｻﾞｷｾｷﾕｶﾞｽ',
  'ｵｶｻﾞｷ ﾖｳｽｹ',
  '岡﨑石油瓦斯(株)',
  '岡﨑 洋介',
  '811-1224',
  '福岡県那珂川市安徳1-13',
  '092-951-3031',
  '1978/10',
  '1957/04',
  '13,000 千円',
  '18 ',
  '6052 燃料小売業',
  '5939 その他の機械器具小売業',
  '0831 一般管工事業',
  'LPガスの販売,ガス器具の販売,住宅設備機器の販売,管工事(8.9%),電気工事,その他工事(0.3%)',
  'Unnamed: 1',
  '(取)岡﨑富子(監)上野二',
  'ENEOSグローブエナジー,リンナイ,ノーリツ,タカラスタンダード',
  '岡﨑洋介(44.6%),西

In [70]:
with open('output.csv', 'w', newline='', encoding='cp932') as file:
    writer = csv.writer(file)
    writer.writerows(pdf_records)

In [41]:
#前田さん共有メモ&TODO
#１行目にcolumnが表示されない
#エクセルに落とすと数字の表示が変わる
#